In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import os
from keras.preprocessing.image import ImageDataGenerator
train_dir = os.path.join('/content/gdrive/MyDrive/faces2/', 'Training')
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
#train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(150, 150),class_mode='binary')

Found 2051 images belonging to 2 classes.


In [6]:
from keras.preprocessing.image import ImageDataGenerator
#train_datagen = ImageDataGenerator(rescale=1./255)
#train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=train_generator = train_datagen.flow_from_directory(train_dir,target_size=(150, 150),class_mode='categorical')
validation_dir = os.path.join('/content/gdrive/MyDrive/faces2/', 'Validation')
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(validation_dir,target_size=(150, 150),class_mode='binary')

Found 819 images belonging to 2 classes.


In [7]:
from keras import layers
from keras import models
model = models.Sequential()
model.add(layers.Conv2D(40, (3, 3), activation='relu',input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


In [8]:
from keras import optimizers
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])

In [9]:
epochs = 20  # Increase the number of epochs for better training
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs
)

Epoch 1/20
65/65 [==============================] - 450s 7s/step - loss: 0.6775 - acc: 0.5597 - val_loss: 0.5276 - val_acc: 0.7680
Epoch 2/20
65/65 [==============================] - 150s 2s/step - loss: 0.6152 - acc: 0.6860 - val_loss: 0.5839 - val_acc: 0.7509
Epoch 3/20
65/65 [==============================] - 145s 2s/step - loss: 0.5904 - acc: 0.7040 - val_loss: 0.5068 - val_acc: 0.7705
Epoch 4/20
65/65 [==============================] - 153s 2s/step - loss: 0.5943 - acc: 0.6880 - val_loss: 0.4980 - val_acc: 0.7790
Epoch 5/20
65/65 [==============================] - 146s 2s/step - loss: 0.5949 - acc: 0.6914 - val_loss: 0.4567 - val_acc: 0.8205
Epoch 6/20
65/65 [==============================] - 148s 2s/step - loss: 0.5646 - acc: 0.7250 - val_loss: 0.4501 - val_acc: 0.8156
Epoch 7/20
65/65 [==============================] - 151s 2s/step - loss: 0.5687 - acc: 0.7148 - val_loss: 0.5050 - val_acc: 0.7521
Epoch 8/20
65/65 [==============================] - 153s 2s/step - loss: 0.5572 - a

In [2]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

NameError: ignored

In [ ]:
import os
import cv2  # OpenCV for image resizing
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

# Define the path to the image
test_dir = '/content/gdrive/MyDrive/faces2/Validation/male/079591.jpg.jpg'

# Load the image using OpenCV and resize it to the desired target size
target_size = (150, 150)
img = cv2.imread(test_dir)
img = cv2.resize(img, target_size)

# Rescale the image to the range [0, 1]
img = img / 255.0

# Create a single-image generator
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow(
    x=np.array([img]),  # Pass the resized image as a NumPy array
    y=None,  # No labels needed for inference
    batch_size=1,  # Set batch size to 1 for a single image
    shuffle=False,  # No need to shuffle for inference
    # You can add more image augmentation options here if needed
)

# Make predictions using your model
y_pred = model.predict(test_generator)

# The 'predictions' variable now contains the model's predictions for the single image
y_pred_classes = (y_pred > 0.5).astype(int)
print(y_pred_classes)

In [29]:
# Evaluate the model on the test set
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150,150),
    class_mode='binary'
)

Found 819 images belonging to 2 classes.


In [30]:
# Assuming y_pred is the output of your model's prediction
y_pred = model.predict(test_generator)

# Convert probabilities to class labels (assuming binary classification)
y_pred_classes = (y_pred > 0.5).astype(int)

# Ensure y_pred_classes has the same number of samples as your test data
assert y_pred_classes.shape[0] == test_generator.samples


26/26 [==============================] - 11s 403ms/step


In [32]:
from sklearn.metrics import classification_report
report = classification_report(test_generator.classes, y_pred_classes)
print(report)

              precision    recall  f1-score   support

           0       0.49      0.61      0.54       408
           1       0.49      0.37      0.43       411

    accuracy                           0.49       819
   macro avg       0.49      0.49      0.49       819
weighted avg       0.49      0.49      0.48       819



In [3]:
# Assuming you have already trained your Keras model and it's stored in a variable 'model'
import pickle

# Save the model to a file using pickle
with open('model_CNN_Faces.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)


NameError: ignored

In [37]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(test_generator.classes, y_pred_classes)
print("Confusion Matrix:")
print(confusion)


Confusion Matrix:
[[249 159]
 [257 154]]
